In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
import pickle

import cudf, cuml, cupy
from cuml.neighbors import NearestNeighbors

from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input
# from tensorflow.keras.applications.efficientnet import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_df = cudf.read_csv('../input/shopee-product-matching/train.csv')
test_df = cudf.read_csv('../input/shopee-product-matching/test.csv')

In [ ]:
# # LOADING TRAIN IMAGE ENCODINGS
# with open("../input/shopeetemp/xcept_0.txt", "rb") as fp:   # Unpickling
#     encs = pickle.load(fp)
# img_knn = NearestNeighbors(n_neighbors=50)
# img_knn.fit(np.array(encs))

In [ ]:
model = load_model('../input/shopee-infer0/xception.h5')

In [ ]:
data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

generator = data_gen.flow_from_dataframe(
        test_df['image'].to_frame().to_pandas(),
        x_col = 'image',
        y_col = None,
        directory = '../input/shopee-product-matching/test_images/',
        target_size=(299, 299),
        batch_size=100,
        shuffle = False,
        class_mode=None)

In [ ]:
q = time.time()
c = 0
d = generator.next()
num = d.shape[0]
c = c + num    
imgs_t = model.predict(d)

eps = int(len(test_df)//100) #+ np.ceil((len(test_df)%1000)/1000))
for ep in range(eps):
    d = generator.next()
    num = d.shape[0]
    c = c + num    
    t = model.predict(d)
    imgs_t = np.concatenate([imgs_t,t])
    del d
    del t
print(time.time()-q)

if len(test_df)>3:
    n = 50
else:
    n=3
img_knn = NearestNeighbors(n_neighbors=n)
img_knn.fit(imgs_t)

In [ ]:
q = time.time()
c = 0
dic = {}
for idx,row in test_df.to_pandas().iterrows():
    img_distances, img_indices = img_knn.kneighbors(imgs_t[idx].reshape((1,2048)))
    l = []
    for k in range(49):
        if img_distances[0,k]<7.7:
            l.append(test_df['posting_id'][img_indices[0,k]])
        else:
            break
    l = [row['posting_id']] + l
    l = np.unique(l)
    dic[row['posting_id']] = " ".join(el for el in l[:50])
print(time.time()-q)

In [ ]:
submission_df = pd.DataFrame(dic.items(),columns=['posting_id','matches'])
submission_df.to_csv("/kaggle/working/submission.csv",index=False)


In [ ]:
!cat submission.csv

In [ ]:
submission_df